# LLama Chat 템플릿을 활용한 Supervised-Fine tuning

## 01. Install packages

In [2]:
!pip install -Uq transformers bitsandbytes datasets

In [3]:
!nvidia-smi

Thu Dec  5 01:51:22 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   45C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [6]:
import torch
from datasets import load_dataset
from transformers import AutoModelForCausalLM, AutoTokenizer

from transformers import Trainer, TrainingArguments

In [7]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.bfloat16).to('cuda')
tokenizer = AutoTokenizer.from_pretrained(model_name)

## 02. Test the foundation LLM (너 한국어 좀 하니?)

In [8]:
prompt = """철수가 연필 12개를 가지고 있었는데 영희가 절반을 가져갔으면 철수에게 남은 연필의 갯수는?"""
messages = [
    {"role": "user", "content": prompt},
]
tokenized_messages = tokenizer.apply_chat_template(messages, return_tensors="pt").to('cuda')
output = model.generate(tokenized_messages, max_new_tokens=512)
print(tokenizer.decode(output[0]))

<|user|>
철수가 연필 12개를 가지고 있었는데 영희가 절반을 가져갔으면 철수에게 남은 연필의 갯수는?</s> 
<|assistant|>
영희가 절반을 가져갔으므로, 철수는 철수가 연필 12개를 가지고 있었는데 영희가 절반을 가져갔으면 철수에게 남은 연필의 갯수는 12개가 됩니다.</s>


In [9]:
tokenized_messages

tensor([[  529, 29989,  1792, 29989, 29958,    13,   239,   181,   163, 30970,
         30903, 29871, 31285,   240,   152,   135, 29871, 29896, 29906, 31789,
         31517, 29871, 30903, 30811, 31137, 29871,   239,   161,   139,   239,
           154,   139, 31081,   238,   144,   179, 29871, 31288,   240,   160,
           175, 30903, 29871,   239,   163,   139,   238,   179,   155, 31286,
         29871, 30903,   239,   163,   187,   237,   179,   151,   239,   159,
           191, 31747, 29871,   239,   181,   163, 30970, 31054,   237,   181,
           143, 29871, 31754, 31354, 29871, 31285,   240,   152,   135, 30708,
         29871,   237,   179,   178, 30970, 31081, 29973,     2, 29871,    13]],
       device='cuda:0')

In [10]:
tokenizer.decode(tokenized_messages[0])

'<|user|>\n철수가 연필 12개를 가지고 있었는데 영희가 절반을 가져갔으면 철수에게 남은 연필의 갯수는?</s> \n'

### 오~! 한국어 어느정도 하는줄 아는구나! 그럼 기사도 한번 요약해볼래?

In [11]:
prompt = """(부천=연합뉴스) 김상연 기자 = 길에서 주운 신용카드로 금은방을 돌며 돌 반지를 산 60대 남성이 경찰에 붙잡혔다.

경기 부천 오정경찰서는 사기와 점유이탈물횡령 등 혐의로 60대 A씨를 검거했다고 2일 밝혔다.

A씨는 지난달 24일 오후 6시부터 8시까지 부천시 오정구 일대 금은방 3곳에서 분실 신용카드를 이용해 300만원 상당의 돌 반지 4개를 구매한 혐의를 받고 있다.

그는 당일 오후 5시께 서울 강서구 길가에서 카드를 주운 뒤 무인 아이스크림 점포에서 600원을 결제했고 부천으로 이동해 추가로 범행한 것으로 파악됐다.

A씨는 경찰에서 "돌 반지를 처분해 생활비로 쓰려고 했다"고 진술했다.

조사 결과 A씨는 과거 절도 혐의로 재판받았으나 선고된 벌금을 내지 않아 수배 명령이 내려진 상태였다.

경찰은 금은방 폐쇄회로(CC)TV 등을 토대로 피의자 신원을 파악해 서울에 있는 A씨 자택에서 그를 검거했다.

경찰 관계자는 "A씨의 수배 사실을 확인해 우선 검찰로 신병을 넘겼다"며 "조만간 사건도 송치할 계획"이라고 말했다.

위 기사 요약해줘."""
messages = [
    {"role": "user", "content": prompt},
]
tokenized_messages = tokenizer.apply_chat_template(messages, return_tensors="pt").to('cuda')
output = model.generate(tokenized_messages, max_new_tokens=512)
print(tokenizer.decode(output[0]))

<|user|>
(부천=연합뉴스) 김상연 기자 = 길에서 주운 신용카드로 금은방을 돌며 돌 반지를 산 60대 남성이 경찰에 붙잡혔다.

경기 부천 오정경찰서는 사기와 점유이탈물횡령 등 혐의로 60대 A씨를 검거했다고 2일 밝혔다.

A씨는 지난달 24일 오후 6시부터 8시까지 부천시 오정구 일대 금은방 3곳에서 분실 신용카드를 이용해 300만원 상당의 돌 반지 4개를 구매한 혐의를 받고 있다.

그는 당일 오후 5시께 서울 강서구 길가에서 카드를 주운 뒤 무인 아이스크림 점포에서 600원을 결제했고 부천으로 이동해 추가로 범행한 것으로 파악됐다.

A씨는 경찰에서 "돌 반지를 처분해 생활비로 쓰려고 했다"고 진술했다.

조사 결과 A씨는 과거 절도 혐의로 재판받았으나 선고된 벌금을 내지 않아 수배 명령이 내려진 상태였다.

경찰은 금은방 폐쇄회로(CC)TV 등을 토대로 피의자 신원을 파악해 서울에 있는 A씨 자택에서 그를 검거했다.

경찰 관계자는 "A씨의 수배 사실을 확인해 우선 검찰로 신병을 넘겼다"며 "조만간 사건도 송치할 계획"이라고 말했다.

위 기사 요약해줘.</s> 
<|assistant|>
A 60-year-old man was arrested by the police in South Korea for stealing a credit card worth 300 million won (approximately $260,000) from a convenience store in the city of Busan. The man, who was caught on CCTV footage using a stolen credit card to make purchases at a supermarket and an ice cream parlor, was found to have committed a felony by the police. The man, who was found to have been involved in a previous case of emb

In [14]:
tokenizer.decode(tokenized_messages[0])

'<|user|>\n(부천=연합뉴스) 김상연 기자 = 길에서 주운 신용카드로 금은방을 돌며 돌 반지를 산 60대 남성이 경찰에 붙잡혔다.\n\n경기 부천 오정경찰서는 사기와 점유이탈물횡령 등 혐의로 60대 A씨를 검거했다고 2일 밝혔다.\n\nA씨는 지난달 24일 오후 6시부터 8시까지 부천시 오정구 일대 금은방 3곳에서 분실 신용카드를 이용해 300만원 상당의 돌 반지 4개를 구매한 혐의를 받고 있다.\n\n그는 당일 오후 5시께 서울 강서구 길가에서 카드를 주운 뒤 무인 아이스크림 점포에서 600원을 결제했고 부천으로 이동해 추가로 범행한 것으로 파악됐다.\n\nA씨는 경찰에서 "돌 반지를 처분해 생활비로 쓰려고 했다"고 진술했다.\n\n조사 결과 A씨는 과거 절도 혐의로 재판받았으나 선고된 벌금을 내지 않아 수배 명령이 내려진 상태였다.\n\n경찰은 금은방 폐쇄회로(CC)TV 등을 토대로 피의자 신원을 파악해 서울에 있는 A씨 자택에서 그를 검거했다.\n\n경찰 관계자는 "A씨의 수배 사실을 확인해 우선 검찰로 신병을 넘겼다"며 "조만간 사건도 송치할 계획"이라고 말했다.\n\n위 기사 요약해줘.</s> \n'

> 간단한 질문은 잘 답변하는데 요약은 영어로 답변을하네?



## 03. Supervised Fine Tuning for Korean

### 03-1. 데이터 준비

In [12]:
ds = load_dataset("daekeun-ml/naver-news-summarization-ko",split='train')
ds = ds.shuffle(42)

README.md:   0%|          | 0.00/787 [00:00<?, ?B/s]

train.csv:   0%|          | 0.00/66.3M [00:00<?, ?B/s]

validation.csv:   0%|          | 0.00/7.45M [00:00<?, ?B/s]

test.csv:   0%|          | 0.00/8.17M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/22194 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2466 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2740 [00:00<?, ? examples/s]

In [13]:
ds[0]

{'date': '2022-07-05 11:14:19',
 'category': 'economy',
 'press': '뉴시스 ',
 'title': '메타버스부터 현장학습까지…유한킴벌리 그린캠프 모집',
 'document': '기사내용 요약 400명 모집…온라인과 실제 숲에서도 함께 진행 서울 뉴시스 2022 유한킴벌리 그린캠프 포스터. 사진 유한킴벌리 제공 2022.07.05 photo newsis.com 재판매 및 DB 금지 서울 뉴시스 권안나 기자 유한킴벌리가 숲을 사랑하는 2022 유한킴벌리 그린캠프 참가자 400명을 오는 20일까지 자사 홈페이지를 통해 모집한다고 5일 밝혔다. 1988년부터 34년간 미래 환경 리더 양성을 위해 이어져 온 유한킴벌리의 그린캠프는 올해 가상세계인 메타버스 숲 그린타운과 국립백두대간수목원에서 진행하는 오프라인 프로그램을 함께 준비했다. 숲을 사랑하는 사람이라면 누구나 지원 가능하다. 지원자 중 400명을 선발하여 온라인 프로그램에 초대한다. 우수교육자 15명을 선발해 숲속 현장학습 프로그램도 함께 진행한다. 온라인 참여자 400명 중 우수 참가자 50명에게는 선물이 제공된다. 그중 15명에게는 숲에서 열리는 현장 학습에 참여할 기회가 주어진다. 온라인 프로그램은 내달 1일과 2일 양일에 걸쳐 그린타운 게더타운 에서 진행한다. 교육과 참여 실천까지 이어질 수 있는 다양한 프로그램을 준비했다. 생명다양성재단 대표인 최재천 이화여대 석좌교수의 기후변화 현재와 미래 그리고 우리 강의를 시작으로 국립백두대간수목원 시드볼트 연구센터 이하얀 팀장의 기후위기와 숲속 생물다양성 보존 노력 사회적 기업 윌든 김승현 대표의 꿀벌과 구상나무 그리고 미래 까지 기후 위기 속 숲과 생물 다양성 보존의 필요성에 대해 배울 수 있는 실시간 온라인 강의가 진행된다. 교육 내용을 복습할 수 있는 ‘OX 퀴즈’ ‘그린벨을 울려라 시즌2 ’ 프로그램과 자신의 실천과 배움을 공유하는 ‘숲피커 선서’ ‘나는 숲피커다’ 프로그램도 진행한다. 그 외 숲을 경험

### 03-2. chat_template 형식으로 데이터 준비하기

In [15]:
messages = [
    {"role": "user", "content": "탬플릿 형식은??"},
]
tokenizer.apply_chat_template(messages, tokenize=False)

'<|user|>\n탬플릿 형식은??</s>\n'

In [16]:
chat_template="""<|user|>
{document}

위 기사 요약해줘.</s>
"""

In [17]:
print(chat_template.format_map({"document":ds[0]['document']}))

<|user|>
기사내용 요약 400명 모집…온라인과 실제 숲에서도 함께 진행 서울 뉴시스 2022 유한킴벌리 그린캠프 포스터. 사진 유한킴벌리 제공 2022.07.05 photo newsis.com 재판매 및 DB 금지 서울 뉴시스 권안나 기자 유한킴벌리가 숲을 사랑하는 2022 유한킴벌리 그린캠프 참가자 400명을 오는 20일까지 자사 홈페이지를 통해 모집한다고 5일 밝혔다. 1988년부터 34년간 미래 환경 리더 양성을 위해 이어져 온 유한킴벌리의 그린캠프는 올해 가상세계인 메타버스 숲 그린타운과 국립백두대간수목원에서 진행하는 오프라인 프로그램을 함께 준비했다. 숲을 사랑하는 사람이라면 누구나 지원 가능하다. 지원자 중 400명을 선발하여 온라인 프로그램에 초대한다. 우수교육자 15명을 선발해 숲속 현장학습 프로그램도 함께 진행한다. 온라인 참여자 400명 중 우수 참가자 50명에게는 선물이 제공된다. 그중 15명에게는 숲에서 열리는 현장 학습에 참여할 기회가 주어진다. 온라인 프로그램은 내달 1일과 2일 양일에 걸쳐 그린타운 게더타운 에서 진행한다. 교육과 참여 실천까지 이어질 수 있는 다양한 프로그램을 준비했다. 생명다양성재단 대표인 최재천 이화여대 석좌교수의 기후변화 현재와 미래 그리고 우리 강의를 시작으로 국립백두대간수목원 시드볼트 연구센터 이하얀 팀장의 기후위기와 숲속 생물다양성 보존 노력 사회적 기업 윌든 김승현 대표의 꿀벌과 구상나무 그리고 미래 까지 기후 위기 속 숲과 생물 다양성 보존의 필요성에 대해 배울 수 있는 실시간 온라인 강의가 진행된다. 교육 내용을 복습할 수 있는 ‘OX 퀴즈’ ‘그린벨을 울려라 시즌2 ’ 프로그램과 자신의 실천과 배움을 공유하는 ‘숲피커 선서’ ‘나는 숲피커다’ 프로그램도 진행한다. 그 외 숲을 경험할 수 있는 다양한 게임과 프로그램 등도 기획했다. 숲속 현장학습은 같은달 5일 경북 봉화군에 위치한 국립백두대간수목원에서 열릴 예정이다. 국립백두대간수목원은 국내외 멸종위기종 등 다양한 야생식물씨앗 보존을 위해 설

In [18]:
print("<|assistant|>\n"+ds[0]['summary']+tokenizer.eos_token)

<|assistant|>
유한킴벌리가 숲을 사랑하는 2022 유한킴벌리 그린캠프 참가자 400명을 오는 20일까지 자사 홈페이지를 통해 모집하는데 숲을 사랑하는 사람이라면 누구나 지원 가능하며 메타버스 숲 그린타운과 국립백두대간수목원에서 진행하는 오프라인 프로그램을 함께 준비했다.</s>


In [21]:
messages = [
    {"role": "user", "content": "탬플릿 형식은??"},
    {"role": "assistant", "content": "나도 모르는데 어쩔"},
]
tokenizer.apply_chat_template(messages, tokenize=False)

'<|user|>\n탬플릿 형식은??</s>\n<|assistant|>\n나도 모르는데 어쩔</s>\n'

In [19]:
def collate_fn(examples):
    input_ids_list = []
    labels_list = []
    attention_mask = []

    for example in examples:
        doc_token = tokenizer.encode(chat_template.format_map({"document": example["document"]}), add_special_tokens=False)
        sum_token = tokenizer.encode("<|assistant|>\n" + example["summary"] + tokenizer.eos_token, add_special_tokens=False)
        input_ids = doc_token + sum_token
        labels = [-100] * len(doc_token) + sum_token

        input_ids_list.append(input_ids)
        labels_list.append(labels)
        attention_mask.append([1] * len(input_ids))

    batch = {
        "input_ids": torch.tensor(input_ids_list, dtype=torch.long),
        "attention_mask": torch.tensor(attention_mask, dtype=torch.long),
        "labels": torch.tensor(labels_list, dtype=torch.long)
    }
    return batch

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    per_device_train_batch_size=1,
    remove_unused_columns=False,
    report_to="none",
    optim="adamw_bnb_8bit",
    gradient_checkpointing=True,
    max_steps=35,
    logging_steps=5,
)

In [ ]:
trainer = Trainer(
        model=model,
        data_collator=collate_fn,
        args=training_args,
        train_dataset=ds,
    )

trainer.train()

max_steps is given, it will override any value given in num_train_epochs
Token indices sequence length is longer than the specified maximum sequence length for this model (2659 > 2048). Running this sequence through the model will result in indexing errors
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.


Step,Training Loss
5,2.842500
10,0.265700
15,2.533800
20,1.893800
25,0.695600
30,1.715800
35,0.316700


TrainOutput(global_step=35, training_loss=1.4662482976913451, metrics={'train_runtime': 318.1249, 'train_samples_per_second': 0.11, 'train_steps_per_second': 0.11, 'total_flos': 351003844816896.0, 'train_loss': 1.4662482976913451, 'epoch': 0.0015770027935478057})

# 학습이 잘됬는지 확인하기 위해 위에서 실패했던 기사를 다시 요약시켜보자

In [ ]:
prompt = """(부천=연합뉴스) 김상연 기자 = 길에서 주운 신용카드로 금은방을 돌며 돌 반지를 산 60대 남성이 경찰에 붙잡혔다.

경기 부천 오정경찰서는 사기와 점유이탈물횡령 등 혐의로 60대 A씨를 검거했다고 2일 밝혔다.

A씨는 지난달 24일 오후 6시부터 8시까지 부천시 오정구 일대 금은방 3곳에서 분실 신용카드를 이용해 300만원 상당의 돌 반지 4개를 구매한 혐의를 받고 있다.

그는 당일 오후 5시께 서울 강서구 길가에서 카드를 주운 뒤 무인 아이스크림 점포에서 600원을 결제했고 부천으로 이동해 추가로 범행한 것으로 파악됐다.

A씨는 경찰에서 "돌 반지를 처분해 생활비로 쓰려고 했다"고 진술했다.

조사 결과 A씨는 과거 절도 혐의로 재판받았으나 선고된 벌금을 내지 않아 수배 명령이 내려진 상태였다.

경찰은 금은방 폐쇄회로(CC)TV 등을 토대로 피의자 신원을 파악해 서울에 있는 A씨 자택에서 그를 검거했다.

경찰 관계자는 "A씨의 수배 사실을 확인해 우선 검찰로 신병을 넘겼다"며 "조만간 사건도 송치할 계획"이라고 말했다.

위 기사 요약해줘."""
messages = [
    {"role": "user", "content": prompt},
]
tokenized_messages = tokenizer.apply_chat_template(messages, return_tensors="pt").to('cuda')
output = model.generate(tokenized_messages, max_new_tokens=512)
print(tokenizer.decode(output[0]))

/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:87: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(


<|user|>
(부천=연합뉴스) 김상연 기자 = 길에서 주운 신용카드로 금은방을 돌며 돌 반지를 산 60대 남성이 경찰에 붙잡혔다.

경기 부천 오정경찰서는 사기와 점유이탈물횡령 등 혐의로 60대 A씨를 검거했다고 2일 밝혔다.

A씨는 지난달 24일 오후 6시부터 8시까지 부천시 오정구 일대 금은방 3곳에서 분실 신용카드를 이용해 300만원 상당의 돌 반지 4개를 구매한 혐의를 받고 있다.

그는 당일 오후 5시께 서울 강서구 길가에서 카드를 주운 뒤 무인 아이스크림 점포에서 600원을 결제했고 부천으로 이동해 추가로 범행한 것으로 파악됐다.

A씨는 경찰에서 "돌 반지를 처분해 생활비로 쓰려고 했다"고 진술했다.

조사 결과 A씨는 과거 절도 혐의로 재판받았으나 선고된 벌금을 내지 않아 수배 명령이 내려진 상태였다.

경찰은 금은방 폐쇄회로(CC)TV 등을 토대로 피의자 신원을 파악해 서울에 있는 A씨 자택에서 그를 검거했다.

경찰 관계자는 "A씨의 수배 사실을 확인해 우선 검찰로 신병을 넘겼다"며 "조만간 사건도 송치할 계획"이라고 말했다.

위 기사 요약해줘.</s> 
 <|assistant|>
오후 6시부터 8시까지 부천시 오정구 일대 금은방 3곳에서 분실 신용카드를 이용해 300만원 상당의 돌 반지 4개를 구매한 혐의를 받고 있다.</s>


# 오~! 이제 한국어로 요약을 생성하는구나
# 하지만, 아직 성능이 별론데...

## Batch를 2 이상하려면 collator를 어떻게 구성해야할까?

### Multi-turn으로 구현하려면 어떻게 구성해야할까?